# Using the Drake CSD Bridge

CSDecomp provides some limited support for directly converting drake plants into a csd plant. Currently only revolute, prismatic and fixed joints along with sphere and box collision geometries are supported.

First, let us construct a drake plant!

In [1]:
from pydrake.all import (RobotDiagramBuilder, 
                         StartMeshcat, 
                         LoadModelDirectives,
                         ProcessModelDirectives,
                         ApplyVisualizationConfig,
                         VisualizationConfig,
                         Rgba,
                         SceneGraphCollisionChecker,
                         SceneGraphInspector,
                         QueryObject)

from pycsdecomp import (convert_drake_plant_to_csd_plant,
                        HPolyhedron,
                        UniformSampleInHPolyhedraCuda,
                        CheckCollisionFreeCuda)
import time

In [ ]:
directives_path = '../csdecomp/tests/test_assets/directives/kinova_sens_on_table.yml'
meshcat = StartMeshcat()
rbuilder = RobotDiagramBuilder()
plant= rbuilder.plant()
builder = rbuilder.builder()
parser = rbuilder.parser()
parser.package_map().Add("test_assets", "../csdecomp/tests/test_assets")
parser.package_map().Add("kortex_description", "../csdecomp/tests/test_assets/kinova/kortex_description")
parser.package_map().Add("robotiq_arg85_description", "../csdecomp/tests/test_assets/robotiq")

directives = LoadModelDirectives(directives_path)
models = ProcessModelDirectives(directives, plant, parser)
scene_graph = rbuilder.scene_graph()
config = VisualizationConfig()
config.enable_alpha_sliders = False
config.publish_contacts = False
config.publish_inertia = False
config.default_proximity_color = Rgba(0.8,0,0,0.5)
plant.Finalize()
ApplyVisualizationConfig(config, builder, meshcat=meshcat)
diagram = rbuilder.Build()
diagram_context = diagram.CreateDefaultContext()
diagram.ForcedPublish(diagram_context)
kin_idx = plant.GetModelInstanceByName("kinova")
rob_idx = plant.GetModelInstanceByName("robotiq")
checker = SceneGraphCollisionChecker(model = diagram,
                           robot_model_instances = [kin_idx, rob_idx],
                           edge_step_size = 0.1)
scene_graph_context = scene_graph.GetMyContextFromRoot(diagram_context)
plant_context = plant.GetMyContextFromRoot(diagram_context)
query : QueryObject = scene_graph.get_query_output_port().Eval(scene_graph_context)
inspector : SceneGraphInspector = query.inspector()
meshcat.SetProperty('/drake/proximity', 'visible', True)

Now, let's convert it!

In [3]:
csd_plant = convert_drake_plant_to_csd_plant(plant,
                                             plant_context,
                                             inspector)

Now, let's check that collision checking is working!

In [ ]:
n_samples = 1000000
domain = HPolyhedron()
domain.MakeBox(csd_plant.getPositionLowerLimits(), csd_plant.getPositionUpperLimits())
samples = UniformSampleInHPolyhedraCuda(polyhedra=[domain], 
                                        starting_points=domain.ChebyshevCenter(), 
                                        num_samples_per_hpolyhedron=n_samples, 
                                        mixing_steps=100,
                                        )[0]
res_csd = CheckCollisionFreeCuda(samples, csd_plant.getMinimalPlant())
res_drake = checker.CheckConfigsCollisionFree(samples.T, parallelize=True)

number_differing_results = 0
critical_configs = []
for idx, (rcsd, rdr) in enumerate(zip(res_csd, res_drake)):
    if rcsd!=rdr:
        number_differing_results +=1
        critical_configs.append(samples[:, idx])


In [ ]:
print(f"Number of differing results found in {n_samples} configurations: {number_differing_results}")

for idx in range(10):
    plant.SetPositions(plant_context, samples[:, idx])
    diagram.ForcedPublish(diagram_context)
    print(f"Collision-free {res_csd[idx]}")
    time.sleep(2)